In [1]:
from mts.core.mtserie_dataset import MTSerieDataset
from mts.core.projections import ProjectionAlg
from models.emotion_dataset_controller import *
import matplotlib.pyplot as plt
from mts.core.projections import ProjectionAlg, euclidean_distance_matrix, mds_projection, compute_k_distance_matrixes, compute_distance_matrix
import tensorflow as tf


config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True
tf.keras.backend.set_session(tf.Session(config=config));


import sys
sys.path.insert(1, '/home/texs/Documents/AirQuality/repositories/peax/experiments')


In [2]:
controller = AppController()

In [3]:
# dataset_id = 'emotions_in_music'
dataset_id = 'afew_va'
controller.loadLocalDataset(dataset_id)

True

In [4]:

dataset = controller.datasets[dataset_id]
X = dataset.values()

print(X.shape)
print(X.max())
print(X.min())

(418, 34, 2)
0.95
0.05


In [5]:
from sklearn.preprocessing import MinMaxScaler

def normalizeWindow(data, percentile: float = 99.9):
    cutoff = np.percentile(data, (0, percentile))
    data_norm = np.copy(data)
    data_norm[np.where(data_norm < cutoff[0])] = cutoff[0]
    data_norm[np.where(data_norm > cutoff[1])] = cutoff[1]

    return MinMaxScaler().fit_transform(data_norm)

X = dataset.values()
# X = np.array([  normalizeWindow(np.concatenate([x, np.zeros([1, 2])]) )    for x in X])
# X = np.array([(np.concatenate([x, np.zeros([1, 2])]) )    for x in X])


N, T, D =X.shape
allWindows = np.split(X, 2, axis = 2)
windows1 = allWindows[0]
# print(windows1.shape)
# print(windows2.shape)

In [6]:
windows = windows1
n = windows.shape[0]
train_index = int(n* 0.6)
test_index = train_index + int(n* 0.2)
train = windows[0: train_index]
test = windows[train_index: test_index]
val = windows[test_index: ]
print(train.shape)
print(test.shape)
print(val.shape)

DATA_FILEPATH = f'savedData/data.npy'
with open(DATA_FILEPATH, 'wb') as f:
    np.save(f, train)
    np.save(f, test)
    np.save(f, val)


(250, 34, 1)
(83, 34, 1)
(85, 34, 1)


In [7]:
settings = {
  "conv_filters": [
    128,
    192,
    288,
    432
  ],
  "conv_kernels": [
    3,
    5,
    7,
    9
  ],
  "dense_units": [
    1024,
    256
  ],
  "dropouts": [
    0,
    0,
    0,
    0,
    0,
    0
  ],
  "embedding": 25,
  "reg_lambda": 0,
  "optimizer": "adadelta",
  "learning_rate": 1.0,
  "learning_rate_decay": 0.001,
  "loss": "bce",
  "metrics": [],
  "batch_norm": [
    False,
    False,
    False,
    False,
    False,
    False
  ],
  "batch_norm_input": False,
}

In [8]:
DATA_FILEPATH = f'savedData/data.npy'
with open(DATA_FILEPATH, 'rb') as f:
    train_data = np.load(f, allow_pickle=True)
    test_data = np.load(f, allow_pickle=True)
    val_data = np.load(f, allow_pickle=True)


In [1]:
# from nn_model import create_model

# _, _, model = create_model(input_dim=T, n_features=D, summary=True)

In [ ]:
model.fit(train_data,train_data)

In [20]:
from air_quality import trainAutoencoder
# import importlib
# importlib.reload(air_quality)

encoder, decoder, autoencoder, history = trainAutoencoder(
    train_data,
    test_data, 
    val_data, 
    model_name = 'savedData/test.h5', 
    window_size = T,
    batch_size = 100,
    n_epochs = 120,
    settings = settings,
)

/home/texs/anaconda3/envs/peax3/lib/python3.7/site-packages/mass_ts/_mass_ts.py:18: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  'GPU support will not work. You must pip install mass-ts[gpu].')
Using TensorFlow backend.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 250 samples, validate on 85 samples
Epoch 1/120
250/250 [==============================] - 2s 8ms/step - loss: 0.6909 - val_loss: 0.6836
Epoch 2/120
250/250 [==============================] - 0s 176us/step - loss: 0.6796 - val_loss: 0.6728
Epoch 3/120
250/250 [==============================] - 0s 168us/step - loss: 0.6682 - val_loss: 0.6668
Epoch 4/120
250/250 [==============================] - 0s 177us/step - loss: 0.6628 - val_loss: 0.6662
Epoch 5/120
250/250 [==============================] - 0s 157us/step - loss: 0.6620 - val_loss: 0.6669
Epoch 6/120
250/250 [==============================] - 0s 188us/step - loss: 0.6621 - val_loss: 0.6663
Epoch 7/120
250/250 [==============================] - 0s 162us/step - loss: 0.6619 - val_loss: 0.6665
Epoch 8/120
250/250 [==============================] - 0s 158us/step - loss: 0.6617 - val_loss: 0.6659
Epoch 9/120
250/250 [==============